In [3]:
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_API_ENDPOINT"), 
  api_key=os.getenv("AZURE_API_KEY"),  
  api_version=os.getenv("AZURE_API_VERSION")
)

In [74]:
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = client.chat.completions.create(
    model="GPT-4-32k-playground",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
    # stream=True
)

In [73]:
chunks=[]
for chunk in response:
    chunks.append(chunk)

In [70]:
chunks=[]
for chunk in response:
    chunks.append(chunk)

ChatCompletionChunk(id='', choices=[], created=0, model='', object='', system_fingerprint=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
ChatCompletionChunk(id='chatcmpl-8rRusUcRTib2OTQIZpbzaFU1PnJYX', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=[ChoiceDeltaToolCall(index=0, id='call_N4RVwuBOh0kma8Ugb3nh7PaV', function=ChoiceDeltaToolCallFunction(arguments='', name='get_current_weather'), type='function')]), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1707749386, model='gpt-4-32k', object='chat.completion.chunk', system_fingerprint=None)
ChatCompletionChunk(id='chatcmpl-8rRusUcRTib2OTQIZpbzaFU1PnJYX', choices=[Choice(delta=ChoiceDelta(content=None, function_call=

In [126]:
from openai.types.chat import ChatCompletion, ChatCompletionChunk, ChatCompletionMessage, ChatCompletionMessageToolCall
from openai.types.chat.chat_completion import Choice
from openai.types.chat.chat_completion_message_tool_call import Function

if chunks[-1].choices[0].finish_reason == 'tool_calls':
    tool_calls=[chunk.choices[0].delta.tool_calls[0] for chunk in chunks[1:-1]]
    tool_call_id=tool_calls[0].id
    tool_call_name=tool_calls[0].function.name
    tool_call_type=tool_calls[0].type
    tool_call_arguments=''
    for chunk in tool_calls[1:]:
        tool_call_arguments+=chunk.function.arguments

    ChatCompletion(id=chunks[-1].id, created=chunks[-1].created, model=chunks[-1].model, object='chat.completion', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id=tool_call_id, function=Function(arguments=tool_call_arguments, name=tool_call_name), type=tool_call_type)]))])
elif chunks[-1].choices[0].finish_reason == 'stop':
    stop_content=''.join(filter(None, [chunk.choices[0].delta.content for chunk in chunks[1:]]))

    ChatCompletion(id=chunks[-1].id, created=chunks[-1].created, model=chunks[-1].model, object='chat.completion', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=stop_content, role='assistant', function_call=None, tool_calls=None))])

SyntaxError: invalid syntax (4004616433.py, line 15)

In [127]:
messages = [{"role": "user", "content": "say hi 3 times"}]
response = client.chat.completions.create(
    model="GPT-4-32k-playground",
    messages=messages,
    # stream=True
)

In [110]:
chunks = []
for chunk in response:
    chunks.append(chunk)

In [112]:
chunks

[ChatCompletionChunk(id='', choices=[], created=0, model='', object='', system_fingerprint=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}]),
 ChatCompletionChunk(id='chatcmpl-8rSFvkssHVi0y85RsgMqyRQmjsP3I', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1707750691, model='gpt-4-32k', object='chat.completion.chunk', system_fingerprint=None),
 ChatCompletionChunk(id='chatcmpl-8rSFvkssHVi0y85RsgMqyRQmjsP3I', choices=[Choice(delta=ChoiceDelta(content='Hi', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 's

In [125]:
if chunks[-1].choices[0].finish_reason == 'stop':
    stop_content=''.join(filter(None, [chunk.choices[0].delta.content for chunk in chunks[1:]]))

ChatCompletion(id=chunks[-1].id, created=chunks[-1].created, model=chunks[-1].model, object='chat.completion', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=stop_content, role='assistant', function_call=None, tool_calls=None))])

ChatCompletion(id='chatcmpl-8rSFvkssHVi0y85RsgMqyRQmjsP3I', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hi, hi, hi.', role='assistant', function_call=None, tool_calls=None))], created=1707750691, model='gpt-4-32k', object='chat.completion', system_fingerprint=None, usage=None)

In [114]:
response

ChatCompletion(id='chatcmpl-8rSGSf72xDAytYmS2s4O092vOJZ3w', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hi, hi, hi.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1707750724, model='gpt-4-32k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=12, total_tokens=18), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

In [40]:
from pydantic import TypeAdapter
from typing import List
from openai.types.chat import ChatCompletion, ChatCompletionChunk
from openai.types.chat.chat_completion import Choice

def merge_chunks_to_completion(chunks: List[ChatCompletionChunk]) -> ChatCompletion:
    merged_choices = []
    finish_reason = None

    new_choice = Choice()
    merged_choices.append(new_choice.delta)

    for chunk in chunks:
        for choice in chunk.choices:
            if choice.delta.content is not None:
                # If there's content, we're in a streaming scenario
                if 'content' not in merged_choices[-1]:
                    merged_choices[-1]['content'] = ''
                merged_choices[-1]['content'] += choice.delta.content
            if choice.delta.tool_calls:
                # If there are tool calls, we're in a tool_calls scenario
                merged_choices[-1]['tool_calls'] = choice.delta.tool_calls
                finish_reason = 'tool_calls'
            if choice.finish_reason:
                # If there's a finish_reason, it's the end of the stream
                finish_reason = choice.finish_reason

    # Create the final ChatCompletion object
    final_completion = ChatCompletion(
        # ... (other fields)
        choices=TypeAdapter(List[Choice]).validate_python(merged_choices),
        finish_reason=finish_reason
    )

    return final_completion

In [41]:
merge_chunks_to_completion(chunks)

IndexError: list index out of range